In [1]:
import io
import json
import os
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload


# ======================================================
# 1) Credenciales (Colab usa MI_JSON desde userdata; fuera de Colab usa MI_JSON env)
# ======================================================
def get_credentials() -> Credentials:
    """
    - En Colab: usa MI_JSON desde google.colab.userdata (NO os.environ)
    - En GitHub/local: usa variable de entorno MI_JSON
    """
    info: Dict

    try:
        from google.colab import userdata  # type: ignore

        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado en Colab userdata")
        info = json.loads(mi_json)
        print("Entorno detectado: Google Colab")
    except Exception:
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado como variable de entorno (GitHub/local)")
        info = json.loads(mi_json)
        print("Entorno detectado: GitHub / local")

    return Credentials.from_service_account_info(
        info,
        scopes=[
            "https://www.googleapis.com/auth/drive.readonly",
            "https://www.googleapis.com/auth/spreadsheets.readonly",
        ],
    )


creds = get_credentials()
drive_service = build("drive", "v3", credentials=creds)


# ======================================================
# 2) Helpers: meses en español + parsing del nombre del archivo
# ======================================================
MES_MAP = {
    "ene": 1,
    "feb": 2,
    "mar": 3,
    "abr": 4,
    "may": 5,
    "jun": 6,
    "jul": 7,
    "ago": 8,
    "sep": 9,
    "oct": 10,
    "nov": 11,
    "dic": 12,
}
MES_NOMBRE = {
    1: "Enero",
    2: "Febrero",
    3: "Marzo",
    4: "Abril",
    5: "Mayo",
    6: "Junio",
    7: "Julio",
    8: "Agosto",
    9: "Septiembre",
    10: "Octubre",
    11: "Noviembre",
    12: "Diciembre",
}


def sheet_name_from_date(dt: datetime) -> str:
    """Ej: datetime(2025,12,...) -> 'Diciembre 2025' """
    return f"{MES_NOMBRE[dt.month]} {dt.year}"


def parse_range_from_filename(name: str) -> Optional[Tuple[int, int, int, int]]:
    """
    Espera nombres tipo:
      'Asignaciones de Cartera Ene26-Abr26.xlsx'
      'Asignaciones de Cartera Sep25-Dic25.xlsx'
    Retorna (start_year, start_month, end_year, end_month) o None si no matchea.
    """
    m = re.search(r"([A-Za-z]{3})(\d{2})\s*-\s*([A-Za-z]{3})(\d{2})", name, flags=re.IGNORECASE)
    if not m:
        return None

    m1, y1, m2, y2 = m.group(1).lower(), m.group(2), m.group(3).lower(), m.group(4)
    if m1 not in MES_MAP or m2 not in MES_MAP:
        return None

    start_month = MES_MAP[m1]
    end_month = MES_MAP[m2]
    start_year = 2000 + int(y1)
    end_year = 2000 + int(y2)

    return (start_year, start_month, end_year, end_month)


def month_index(year: int, month: int) -> int:
    """Convierte (year, month) a índice comparable."""
    return year * 12 + month


def file_covers_month(file_range: Tuple[int, int, int, int], target_dt: datetime) -> bool:
    sy, sm, ey, em = file_range
    t = month_index(target_dt.year, target_dt.month)
    a = month_index(sy, sm)
    b = month_index(ey, em)
    return a <= t <= b


def is_df_empty_like(df: Optional[pd.DataFrame]) -> bool:
    """
    Considera 'vacía' si:
    - df es None
    - df tiene 0 filas
    - o todas las filas están completamente NaN
    """
    if df is None or df.shape[0] == 0:
        return True
    return df.dropna(how="all").shape[0] == 0


# ======================================================
# 3) Listar archivos en carpeta y elegir el correcto por mes
# ======================================================
def list_assignment_files_in_folder(folder_id: str) -> List[Dict]:
    """
    Lista archivos en la carpeta cuyo nombre contenga 'Asignaciones de Cartera'
    y devuelve una lista con metadatos: id, name, mimeType, modifiedTime, parsed_range
    """
    q = f"'{folder_id}' in parents and trashed=false and name contains 'Asignaciones de Cartera'"
    files: List[Dict] = []
    page_token = None

    while True:
        resp = (
            drive_service.files()
            .list(q=q, fields="nextPageToken, files(id,name,mimeType,modifiedTime)", pageToken=page_token)
            .execute()
        )

        for f in resp.get("files", []):
            fr = parse_range_from_filename(f.get("name", ""))
            if fr:
                f["parsed_range"] = fr
                files.append(f)

        page_token = resp.get("nextPageToken")
        if not page_token:
            break

    if not files:
        raise ValueError(
            "No encontré archivos 'Asignaciones de Cartera' con rango tipo Ene26-Abr26 dentro de la carpeta."
        )

    return files


def pick_file_for_month(files_meta: List[Dict], target_dt: datetime) -> Dict:
    """
    Escoge el archivo cuya ventana (en el nombre) cubra el mes target_dt.
    Si hay varios, elige el de rango más corto (más específico) y si empatan, el más reciente.
    """
    candidates = []
    for f in files_meta:
        fr = f["parsed_range"]
        if file_covers_month(fr, target_dt):
            sy, sm, ey, em = fr
            span = month_index(ey, em) - month_index(sy, sm)
            candidates.append((span, f.get("modifiedTime", ""), f))

    if candidates:
        candidates.sort(key=lambda x: (x[0], x[1]))  # menor span, luego por modifiedTime asc
        min_span = candidates[0][0]
        same_span = [c for c in candidates if c[0] == min_span]
        same_span.sort(key=lambda x: x[1], reverse=True)  # más reciente primero
        return same_span[0][2]

    t = month_index(target_dt.year, target_dt.month)

    past = []
    for f in files_meta:
        sy, sm, ey, em = f["parsed_range"]
        end_i = month_index(ey, em)
        if end_i <= t:
            past.append((end_i, f.get("modifiedTime", ""), f))
    if past:
        past.sort(key=lambda x: (x[0], x[1]), reverse=True)
        return past[0][2]

    future = []
    for f in files_meta:
        sy, sm, ey, em = f["parsed_range"]
        start_i = month_index(sy, sm)
        if start_i >= t:
            future.append((start_i, f.get("modifiedTime", ""), f))
    if future:
        future.sort(key=lambda x: (x[0], x[1]))
        return future[0][2]

    raise ValueError("No se pudo escoger un archivo por fecha (revisa nombres/rangos).")


# ======================================================
# 4) Descargar archivo (Google Sheets o Excel) a memoria
# ======================================================
def download_file_to_buffer(file_id: str, mime_type: str) -> io.BytesIO:
    buffer = io.BytesIO()

    if mime_type == "application/vnd.google-apps.spreadsheet":
        request = drive_service.files().export_media(
            fileId=file_id,
            mimeType="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        )
    else:
        request = drive_service.files().get_media(fileId=file_id)

    downloader = MediaIoBaseDownload(buffer, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()

    buffer.seek(0)
    return buffer


# ======================================================
# 5) Intentar leer hoja del mes target; si está vacía -> retroceder mes a mes
# ======================================================
def shift_month(dt: datetime, n: int) -> datetime:
    """Mueve dt n meses (n puede ser negativo)."""
    y = dt.year + (dt.month - 1 + n) // 12
    m = (dt.month - 1 + n) % 12 + 1
    return datetime(y, m, 1)


def load_assignment_base_from_folder(
    folder_id: str, max_back_months: int = 24
) -> Tuple[pd.DataFrame, Dict, str]:
    """
    Busca base del mes actual; si no existe o está vacía,
    busca mes anterior (incluyendo cambio de archivo si aplica).
    Retorna: (df, file_meta, sheet_name_usada)
    """
    files_meta = list_assignment_files_in_folder(folder_id)
    today = datetime.today()

    last_error: Optional[Exception] = None

    for back in range(0, max_back_months + 1):
        target_dt = shift_month(today, -back)
        target_sheet = sheet_name_from_date(target_dt)
        chosen = pick_file_for_month(files_meta, target_dt)

        try:
            buffer = download_file_to_buffer(chosen["id"], chosen["mimeType"])
            df = pd.read_excel(buffer, sheet_name=target_sheet, engine="openpyxl")

            if is_df_empty_like(df):
                print(f"🟡 {target_sheet} encontrado pero vacío en: {chosen['name']} -> probando mes anterior...")
                continue

            print("✅ Base encontrada")
            print(f"   Archivo: {chosen['name']}")
            print(f"   Hoja:    {target_sheet}")
            return df, chosen, target_sheet

        except Exception as e:
            last_error = e
            print(
                f"🟠 No se pudo usar {target_sheet} en {chosen['name']} ({type(e).__name__}) -> probando mes anterior..."
            )

    raise RuntimeError(
        f"No encontré una hoja válida en los últimos {max_back_months} meses. "
        f"Último error: {repr(last_error)}"
    )


# ======================================================
# 6) EJECUCIÓN
# ======================================================
FOLDER_ID = "1cf2p3R7iM0xowAt4muEruDwxZoZqD_jB"

df, meta_file, sheet_used = load_assignment_base_from_folder(
    folder_id=FOLDER_ID,
    max_back_months=24,
)

df.head()

Entorno detectado: GitHub / local


✅ Base encontrada
   Archivo: Asignaciones de Cartera Ene26-Abr26.xlsx
   Hoja:    Febrero 2026


,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,Deuda Resuelve,DBT,...,Bucket,Negociador,Tipo Elegible,PotCred_num,PotCred_cat,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level
0,3106653868,3763762,Jorge Luis Camargo Fontalvo,899359,jlc5905@gmail.com,6,CREDIVALORES,401093039,3996300.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,1176.0,Atrasado,2025-01-21,380.0,Prioridad 1
1,3106653868,3763762,Jorge Luis Camargo Fontalvo,899360,jlc5905@gmail.com,6,Serfinanza,543000000,5763450.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,NaN,Al día,2025-01-21,380.0,Prioridad 2
2,3173740082,1034308242,Nailen Josefina Bahamon Sandoval,1255027,nai.bs@hotmail.com,1,Falabella,300002000000,13506762.0,17310912.0,...,0,Lina Maria Peña Gordillo,NaN,NaN,NaN,0.0,Al día,2024-10-25,468.0,Prioridad 1
3,3224821336,1093755862,Diego Andres Delgado Galvis,1275933,diegodlgd90@gmail.com,1,Davivienda,402516857,10257450.0,10257450.0,...,4,Lina Maria Peña Gordillo,NaN,NaN,NaN,196.0,Atrasado,NaT,NaN,Prioridad 2
4,3167086550,37826977,Luz Mery Quintero Blanco,1297740,luzmb.1225@gmail.com,4,Serfinanza,542060188,8083900.0,91913600.0,...,2,Angie Natalia Borda Soler,NaN,NaN,NaN,483.0,Atrasado,2024-04-30,646.0,Prioridad 1


In [2]:
df = df.rename(columns={"Deuda Resuelve": "D_BRAVO"})

In [3]:
df.columns

Index(['Referencia', 'Cedula', 'Nombre del cliente', 'Id deuda', 'correo',
       'Deudas Activas', 'Banco', 'Número de Crédito', 'D_BRAVO', 'DBT',
       'Meses de atraso', 'Dias de Atraso', 'Apartado Mensual', 'Fecha inicio',
       'vehiculo', 'Meses en el Programa', 'tipo_cliente', 'Comisión Mensual',
       'Tipo de cobro', 'CE', 'Estado Deuda', 'estado_novacion', 'Fecha PL',
       'sub_estado_deuda', 'ID_reparadora', 'estado_reparadora',
       'sub_estado_reparadora', 'dias_mora_ingreso', 'ultima_actividad',
       'PB_PL', 'estado_estructuracion', 'estado_flujo_liquidacion',
       'Ahorro total', 'Ahorro medio', 'Por cobrar', 'ultimo contacto',
       'fecha mensaje', 'Descuento', 'Mora_estructurado', 'MORA_CREDITO',
       'Potencial Credito', 'ultimo Pab', 'ultima act', 'Tipo de Liquidacion',
       'BANCOS_ESTANDAR', 'Clasificación BANCO', 'Apartados 30%', 'Plazo',
       'Estructurable', 'Potencial', 'Bucket', 'Negociador', 'Tipo Elegible',
       'PotCred_num', 'PotCred_

In [4]:
df

,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,D_BRAVO,DBT,...,Bucket,Negociador,Tipo Elegible,PotCred_num,PotCred_cat,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level
0,3106653868,3763762,Jorge Luis Camargo Fontalvo,899359,jlc5905@gmail.com,6,CREDIVALORES,401093039,3996300.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,1176.0,Atrasado,2025-01-21,380.0,Prioridad 1
1,3106653868,3763762,Jorge Luis Camargo Fontalvo,899360,jlc5905@gmail.com,6,Serfinanza,543000000,5763450.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,NaN,Al día,2025-01-21,380.0,Prioridad 2
2,3173740082,1034308242,Nailen Josefina Bahamon Sandoval,1255027,nai.bs@hotmail.com,1,Falabella,300002000000,13506762.0,17310912.0,...,0,Lina Maria Peña Gordillo,NaN,NaN,NaN,0.0,Al día,2024-10-25,468.0,Prioridad 1
3,3224821336,1093755862,Diego Andres Delgado Galvis,1275933,diegodlgd90@gmail.com,1,Davivienda,402516857,10257450.0,10257450.0,...,4,Lina Maria Peña Gordillo,NaN,NaN,NaN,196.0,Atrasado,NaT,NaN,Prioridad 2
4,3167086550,37826977,Luz Mery Quintero Blanco,1297740,luzmb.1225@gmail.com,4,Serfinanza,542060188,8083900.0,91913600.0,...,2,Angie Natalia Borda Soler,NaN,NaN,NaN,483.0,Atrasado,2024-04-30,646.0,Prioridad 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41443,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024433,ale.salo.sastoque@gmail.com,5,Serfinanza,543280552,3756500.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2
41444,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024432,ale.salo.sastoque@gmail.com,5,BBVA,620741,948150.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2
41445,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024430,ale.salo.sastoque@gmail.com,5,Sistecredito,82-000907,199500.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2
41446,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024431,ale.salo.sastoque@gmail.com,5,Sistecredito,35-000261,741300.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2


#Saldos Actualizados

In [5]:
"""
Lee 3 hojas de un Google Sheets con encabezados "corridos" (no en la fila 1),
extrae REFERENCIA y SALDO, agrega columna Vehiculo (POWWI/SKANDIA/COINK),
y concatena todo en un solo DataFrame.

✅ Funciona en:
- Google Colab (usa secreto MI_JSON)
- GitHub / local (usa env MI_JSON o GOOGLE_APPLICATION_CREDENTIALS o service_account.json)

Requisitos:
pip install gspread google-auth pandas
"""

import os
import json
import numpy as np
import pandas as pd
from typing import Union, Dict, Any

import gspread
from google.oauth2.service_account import Credentials


# =========================
# 1) CONFIG
# =========================
SPREADSHEET_ID = "1mvxPdnyp5ip_0Lqyf6qy09BAtX323PF2Yc5-qGoukeU"

SHEETS = [
    ("SALDOS POWWI", "POWWI"),
    ("SALDOS SKANDIA", "SKANDIA"),
    ("SALDOS COINK", "COINK"),
]

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]


# =========================
# 2) CREDENCIALES (Colab / GitHub / Local)
# =========================
def _coerce_to_dict(value: Union[str, dict]) -> Dict[str, Any]:
    """Convierte string JSON o dict a dict."""
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        v = value.strip()

        # Si el secreto/ENV viene con una capa extra de comillas
        if (v.startswith("'") and v.endswith("'")) or (v.startswith('"') and v.endswith('"')):
            # Intentamos quitar solo UNA capa (sin dañar JSON normal)
            try:
                maybe = v[1:-1]
                # Si eso parece JSON, úsalo
                if maybe.strip().startswith("{") and maybe.strip().endswith("}"):
                    v = maybe
            except Exception:
                pass

        return json.loads(v)
    raise TypeError(f"Credenciales en formato no soportado: {type(value)}")


def load_service_account_info() -> Dict[str, Any]:
    """
    Devuelve el dict JSON del service account desde cualquiera de estas fuentes (en orden):
    1) Google Colab secret: MI_JSON (userdata.get)
    2) Variable de entorno: MI_JSON (string JSON)
    3) Variable de entorno: GOOGLE_APPLICATION_CREDENTIALS (path a .json)
    4) Archivo local: service_account.json (si existe)
    """
    # (1) Colab secret
    try:
        from google.colab import userdata  # type: ignore

        mi_json = userdata.get("MI_JSON")
        if mi_json is not None:
            return _coerce_to_dict(mi_json)
    except Exception:
        pass  # no es colab

    # (2) Env var MI_JSON
    env_mi_json = os.getenv("MI_JSON")
    if env_mi_json:
        return _coerce_to_dict(env_mi_json)

    # (3) Path vía GOOGLE_APPLICATION_CREDENTIALS
    gac = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if gac and os.path.exists(gac):
        with open(gac, "r", encoding="utf-8") as f:
            return json.load(f)

    # (4) Archivo local por defecto
    default_path = "service_account.json"
    if os.path.exists(default_path):
        with open(default_path, "r", encoding="utf-8") as f:
            return json.load(f)

    raise FileNotFoundError(
        "No encontré credenciales. Usa UNA de estas opciones:\n"
        "- Colab: guarda el secreto MI_JSON\n"
        "- GitHub/Local: export MI_JSON='{\"type\":...}'\n"
        "- GitHub/Local: export GOOGLE_APPLICATION_CREDENTIALS='/ruta/key.json'\n"
        "- GitHub/Local: crea service_account.json en el proyecto"
    )


def get_gspread_client() -> gspread.Client:
    info = load_service_account_info()
    creds = Credentials.from_service_account_info(info, scopes=SCOPES)
    return gspread.authorize(creds)


# =========================
# 3) HELPERS: HEADER CORRIDO + SALDO A NÚMERO (CORREGIDO)
# =========================
def _norm(x) -> str:
    return str(x).strip().upper() if x is not None else ""


def _to_number_col(s: pd.Series) -> pd.Series:
    """
    Convierte SALDO desde textos como:
    - 1.057.428         -> 1057428
    - 353.281           -> 353281
    - $5.019.407.320,00 -> 5019407320.00
    - 2723949           -> 2723949
    """
    s = s.astype(str).str.strip()

    # quita $ y espacios
    s = s.str.replace(r"[\$\s]", "", regex=True)

    has_comma = s.str.contains(",", na=False)

    # Si hay coma: formato ES -> miles '.' y decimal ','
    s_comma = (
        s.str.replace(".", "", regex=False)
         .str.replace(",", ".", regex=False)
    )

    # Si NO hay coma: '.' se asume miles (se quita)
    s_nocomma = s.str.replace(".", "", regex=False)

    cleaned = np.where(has_comma, s_comma, s_nocomma)
    return pd.to_numeric(cleaned, errors="coerce")


def read_sheet_with_shifted_header(
    gc: gspread.Client,
    spreadsheet_id: str,
    worksheet_name: str,
    vehiculo_value: str,
    max_scan_rows: int = 300,
) -> pd.DataFrame:
    """
    Lee una hoja donde el header real NO está en la fila 1.
    Detecta la fila que contiene 'REFERENCIA' y 'SALDO', arma el DF desde ahí.
    Devuelve columnas: REFERENCIA, SALDO, Vehiculo
    """
    sh = gc.open_by_key(spreadsheet_id)
    ws = sh.worksheet(worksheet_name)
    values = ws.get_all_values()
    raw = pd.DataFrame(values)

    header_row_idx = None
    for i in range(min(len(raw), max_scan_rows)):
        row = raw.iloc[i].map(_norm).tolist()
        if "REFERENCIA" in row and "SALDO" in row:
            header_row_idx = i
            break

    if header_row_idx is None:
        raise ValueError(
            f"No pude encontrar una fila de encabezados con 'REFERENCIA' y 'SALDO' en: {worksheet_name}"
        )

    headers = raw.iloc[header_row_idx].map(lambda x: str(x).strip()).tolist()
    df = raw.iloc[header_row_idx + 1 :].copy()
    df.columns = [h.strip() for h in headers]

    # Asegura columnas REFERENCIA y SALDO aunque vengan con case raro
    cols_upper = {c.upper(): c for c in df.columns}
    if "REFERENCIA" not in df.columns and "REFERENCIA" in cols_upper:
        df.rename(columns={cols_upper["REFERENCIA"]: "REFERENCIA"}, inplace=True)
    if "SALDO" not in df.columns and "SALDO" in cols_upper:
        df.rename(columns={cols_upper["SALDO"]: "SALDO"}, inplace=True)

    if "REFERENCIA" not in df.columns or "SALDO" not in df.columns:
        raise ValueError(
            f"En {worksheet_name} detecté el header pero no quedaron columnas 'REFERENCIA' y 'SALDO'. "
            f"Columnas encontradas: {list(df.columns)}"
        )

    df = df[["REFERENCIA", "SALDO"]].copy()

    # Limpieza básica
    df["REFERENCIA"] = df["REFERENCIA"].astype(str).str.strip()
    df = df[df["REFERENCIA"].ne("")].copy()

    df["SALDO"] = _to_number_col(df["SALDO"])
    df["Vehiculo"] = vehiculo_value

    return df.reset_index(drop=True)


# =========================
# 4) MAIN: LEE + CONCATENA + PRUEBAS RÁPIDAS
# =========================
def build_df_saldos_concat() -> pd.DataFrame:
    gc = get_gspread_client()

    dfs = []
    for sheet_name, vehiculo in SHEETS:
        df_i = read_sheet_with_shifted_header(
            gc=gc,
            spreadsheet_id=SPREADSHEET_ID,
            worksheet_name=sheet_name,
            vehiculo_value=vehiculo,
        )
        dfs.append(df_i)

    df_saldos_concat = pd.concat(dfs, ignore_index=True)[["REFERENCIA", "SALDO", "Vehiculo"]]
    return df_saldos_concat


if __name__ == "__main__":
    df_saldos_concat = build_df_saldos_concat()

    print("✅ Listo. Shape:", df_saldos_concat.shape)
    print(df_saldos_concat.head(15))

    # Chequeo del caso que mencionaste (cambia si quieres)
    test_ref = "3001342492"
    print("\n🔎 Validación referencia", test_ref)
    print(df_saldos_concat.loc[df_saldos_concat["REFERENCIA"].astype(str) == test_ref])

✅ Listo. Shape: (15033, 3)
    REFERENCIA    SALDO Vehiculo
0   3001033679      214    POWWI
1   3001220616      204    POWWI
2   3001342492  1595433    POWWI
3   3001456624        0    POWWI
4   3001542338        0    POWWI
5   3001720277   336910    POWWI
6   3001788198   258643    POWWI
7   3001934505      204    POWWI
8   3001963725  1057036    POWWI
9   3002010121     1504    POWWI
10  3002025530     1521    POWWI
11  3002061945      206    POWWI
12  3002063077  2488595    POWWI
13  3002071472   445279    POWWI
14  3002074569   521227    POWWI

🔎 Validación referencia 3001342492
   REFERENCIA    SALDO Vehiculo
2  3001342492  1595433    POWWI


In [6]:
df_saldos_concat

,REFERENCIA,SALDO,Vehiculo
0,3001033679,214,POWWI
1,3001220616,204,POWWI
2,3001342492,1595433,POWWI
3,3001456624,0,POWWI
4,3001542338,0,POWWI
...,...,...,...
15028,3052664557,0,COINK
15029,3002481664,0,COINK
15030,3059347258,0,COINK
15031,3226855953,0,COINK


In [7]:
import os
import json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# -------- CONFIG --------
SPREADSHEET_ID = "1mvxPdnyp5ip_0Lqyf6qy09BAtX323PF2Yc5-qGoukeU"
SHEET_NAME   = "TOTAL"

SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]

# -------- CREDENCIALES --------
def _coerce_to_dict(value):
    if isinstance(value, dict):
        return value
    if isinstance(value, str):
        v = value.strip()
        if (v.startswith("'") and v.endswith("'")) or (v.startswith('"') and v.endswith('"')):
            maybe = v[1:-1]
            if maybe.strip().startswith("{") and maybe.strip().endswith("}"):
                v = maybe
        return json.loads(v)
    raise TypeError(f"Formato no soportado: {type(value)}")

def load_service_account_info():
    """
    Intenta encontrar credenciales SA JSON desde:
    1) Secrets Colab (MI_JSON)
    2) Env var MI_JSON
    3) GOOGLE_APPLICATION_CREDENTIALS (path)
    4) Archivo local service_account.json
    """
    try:
        from google.colab import userdata
        mi_json = userdata.get("MI_JSON")
        if mi_json:
            return _coerce_to_dict(mi_json)
    except Exception:
        pass

    env = os.getenv("MI_JSON")
    if env:
        return _coerce_to_dict(env)

    gac = os.getenv("GOOGLE_APPLICATION_CREDENTIALS")
    if gac and os.path.exists(gac):
        with open(gac, "r", encoding="utf-8") as f:
            return json.load(f)

    if os.path.exists("service_account.json"):
        with open("service_account.json", "r", encoding="utf-8") as f:
            return json.load(f)

    raise FileNotFoundError("No encontré credenciales SA para Google Sheets")

def get_gspread_client():
    info = load_service_account_info()
    creds = Credentials.from_service_account_info(info, scopes=SCOPES)
    return gspread.authorize(creds)

# -------- LEER HOJA --------
def read_total_sheet():
    gc = get_gspread_client()
    sh = gc.open_by_key(SPREADSHEET_ID)
    ws = sh.worksheet(SHEET_NAME)

    # Lee todo en bruto
    values = ws.get_all_values()
    df = pd.DataFrame(values)

    # si la primera fila YA es encabezado:
    df.columns = df.iloc[0]
    df = df[1:].reset_index(drop=True)

    return df

# -------- EJECUCIÓN --------
df_total = read_total_sheet()

print("Shape:", df_total.shape)
df_total.head(15)

Shape: (17950, 7)


,,REFERENCIA,DOCUMENTO,X COBRAR FLUJOS,X COBRAR FACTURACIÓN,TOTAL,MENSUALIDADES COBRADAS
0,,3213843031,,"$136.769,00","$260.000,00",$396.769,0
1,,3208393999,,"$399.497,00","$0,00",$399.497,0
2,,3107746045,,"$0,00","$380.531,00",$380.531,0
3,,3163143964,,"$145.583,00","$0,00",$145.583,0
4,,1090391063,,"$792.481,00","$0,00",$792.481,0
5,,3016223771,,"$0,00","$316.000,00",$316.000,0
6,,3503946092,,"$0,00","$9.415.663,00",$9.415.663,0
7,,3505950650,,"$200.608,00","$0,00",$200.608,0
8,,3177510366,,"$60.639,00","$0,00",$60.639,0
9,,3144747305,,"$12.500,00","$0,00",$12.500,0


In [8]:
# 1) Normalizar nombres de columnas (por seguridad)
df_saldos_concat.columns = [c.strip().upper() for c in df_saldos_concat.columns]
df_total.columns = [c.strip().upper() for c in df_total.columns]

# 2) Quedarnos solo con lo necesario de df_total
df_total_aux = df_total[["REFERENCIA", "TOTAL"]].copy()

# 3) Merge (left join)
df_saldos_concat = df_saldos_concat.merge(
    df_total_aux,
    on="REFERENCIA",
    how="left"
)

# 4) Renombrar TOTAL → Saldo
df_saldos_concat = df_saldos_concat.rename(columns={"TOTAL": "Saldo"})

df_saldos_concat

,REFERENCIA,SALDO,VEHICULO,Saldo
0,3001033679,214,POWWI,$258.825
1,3001220616,204,POWWI,$201.716
2,3001342492,1595433,POWWI,NaN
3,3001456624,0,POWWI,NaN
4,3001542338,0,POWWI,$396.063
...,...,...,...,...
15028,3052664557,0,COINK,$1.345.126
15029,3002481664,0,COINK,NaN
15030,3059347258,0,COINK,NaN
15031,3226855953,0,COINK,NaN


In [9]:
import numpy as np
import pandas as pd

# 1) Renombrar SALDO -> Ahorro
df_saldos_concat = df_saldos_concat.rename(columns={"SALDO": "Ahorro"})

# 2) Convertir Saldo (string con separador de miles) a numérico
df_saldos_concat["Saldo"] = (
    df_saldos_concat["Saldo"]
        .astype(str)
        .str.strip()
        .str.replace(r"[\$\s]", "", regex=True)   # quita $ y espacios
        .str.replace(".", "", regex=False)        # quita separador de miles
)

df_saldos_concat["Saldo"] = pd.to_numeric(
    df_saldos_concat["Saldo"],
    errors="coerce"
)

df_saldos_concat.head()

,REFERENCIA,Ahorro,VEHICULO,Saldo
0,3001033679,214,POWWI,258825.0
1,3001220616,204,POWWI,201716.0
2,3001342492,1595433,POWWI,NaN
3,3001456624,0,POWWI,NaN
4,3001542338,0,POWWI,396063.0


In [10]:
df_saldos_concat

,REFERENCIA,Ahorro,VEHICULO,Saldo
0,3001033679,214,POWWI,258825.0
1,3001220616,204,POWWI,201716.0
2,3001342492,1595433,POWWI,NaN
3,3001456624,0,POWWI,NaN
4,3001542338,0,POWWI,396063.0
...,...,...,...,...
15028,3052664557,0,COINK,1345126.0
15029,3002481664,0,COINK,NaN
15030,3059347258,0,COINK,NaN
15031,3226855953,0,COINK,NaN


#Ultimo Mensaje actualizado

In [11]:
import os
import requests
import pandas as pd
from io import StringIO
from requests.exceptions import JSONDecodeError

BASE_URL = "https://metabase.resuelve.io"
CARD_ID = 11382


def running_in_colab():
    try:
        import google.colab  # noqa
        return True
    except ImportError:
        return False


def get_mb_credentials():
    user = None
    password = None

    if running_in_colab():
        from google.colab import userdata
        user = userdata.get("MB_USER")
        password = userdata.get("MB_PASS")

    if not user:
        user = os.environ.get("MB_USER")
    if not password:
        password = os.environ.get("MB_PASS")

    if not user or not password:
        raise ValueError("No encontré credenciales de Metabase.")

    return user, password


def create_metabase_session():
    user, password = get_mb_credentials()

    resp = requests.post(
        f"{BASE_URL}/api/session",
        json={"username": user, "password": password},
        timeout=60,
    )
    resp.raise_for_status()

    session_id = resp.json()["id"]
    headers = {"X-Metabase-Session": session_id}
    return headers


def get_card_full(card_id: int) -> pd.DataFrame:
    headers = create_metabase_session()

    try:
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/json",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        rows = resp.json()
        return pd.DataFrame(rows)

    except (JSONDecodeError, ValueError):
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/csv",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.read_csv(StringIO(resp.text))


df_11382 = get_card_full(CARD_ID)
print(df_11382.shape)
df_11382.head()

(15195, 5)


,id,bank_reference,phone_number,country,message_date_utc
0,220847,84703,3102902344,co,2025-06-04T20:06:18Z
1,290723,3158905891,8149232877,us,2025-12-30T00:00:05Z
2,292060,3152396265,3152396265,co,2025-07-24T12:32:54Z
3,294764,3135591292,3135591292,co,2025-04-04T14:40:49Z
4,299269,3144668134,3144668134,co,2026-01-14T15:23:01Z


In [12]:
import pandas as pd

df_11382["message_date_utc"] = (
    pd.to_datetime(df_11382["message_date_utc"], utc=True)  # asegura UTC
      .dt.tz_convert("America/Bogota")                      # pasa a hora Colombia
      .dt.tz_localize(None)                                 # quita info de zona
)

In [13]:
df_11382

,id,bank_reference,phone_number,country,message_date_utc
0,220847,84703,3102902344,co,2025-06-04 15:06:18
1,290723,3158905891,8149232877,us,2025-12-29 19:00:05
2,292060,3152396265,3152396265,co,2025-07-24 07:32:54
3,294764,3135591292,3135591292,co,2025-04-04 09:40:49
4,299269,3144668134,3144668134,co,2026-01-14 10:23:01
...,...,...,...,...,...
15190,609633,3017087516,3017087516,co,2026-02-10 11:42:29
15191,610211,3113340067,3113340067,co,2026-02-09 15:29:53
15192,610670,3216079564,3216079564,co,2026-01-13 13:40:07
15193,616194,3143383806,3143383806,co,2026-02-11 07:51:55


#Estado Reparadora

In [14]:
import os
import requests
import pandas as pd
from io import StringIO
from requests.exceptions import JSONDecodeError

BASE_URL = "https://metabase.resuelve.io"
CARD_ID = 11320


# =========================
# 1) Detectar entorno
# =========================
def running_in_colab():
    try:
        import google.colab  # noqa
        return True
    except ImportError:
        return False


# =========================
# 2) Credenciales Metabase
# =========================
def get_mb_credentials():
    user = None
    password = None

    # Colab: Secrets
    if running_in_colab():
        from google.colab import userdata
        user = userdata.get("MB_USER")
        password = userdata.get("MB_PASS")

    # GitHub / local: env vars
    if not user:
        user = os.environ.get("MB_USER")
    if not password:
        password = os.environ.get("MB_PASS")

    if not user or not password:
        raise ValueError("No encontré credenciales de Metabase (MB_USER / MB_PASS).")

    return user, password


# =========================
# 3) Sesión Metabase
# =========================
def create_metabase_session():
    user, password = get_mb_credentials()

    resp = requests.post(
        f"{BASE_URL}/api/session",
        json={"username": user, "password": password},
        timeout=60,
    )
    resp.raise_for_status()

    session_id = resp.json()["id"]
    return {"X-Metabase-Session": session_id}


# =========================
# 4) Descargar tarjeta
# =========================
def get_card_full(card_id: int) -> pd.DataFrame:
    headers = create_metabase_session()

    try:
        # Intento JSON
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/json",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.DataFrame(resp.json())

    except (JSONDecodeError, ValueError):
        # Fallback CSV
        resp = requests.post(
            f"{BASE_URL}/api/card/{card_id}/query/csv",
            headers=headers,
            json={},
            timeout=600,
        )
        resp.raise_for_status()
        return pd.read_csv(StringIO(resp.text))


# =========================
# 5) Ejecutar + filtrar columnas
# =========================
df_11320 = get_card_full(CARD_ID)

# normalizar nombres de columnas
df_11320.columns = [c.lower() for c in df_11320.columns]

cols_needed = ["referencia", "id_deuda", "estado_deuda", "sub_estado_deuda", "estado_reparadora", "sub_estado_reparadora"]

missing = set(cols_needed) - set(df_11320.columns)
if missing:
    raise ValueError(f"Faltan columnas requeridas en el DataFrame: {missing}")

df_11320 = df_11320[cols_needed].copy()

print("✅ DataFrame listo:", df_11320.shape)
df_11320.head()

✅ DataFrame listo: (49448, 6)


,referencia,id_deuda,estado_deuda,sub_estado_deuda,estado_reparadora,sub_estado_reparadora
0,3213797430,1231949,paid_portfolio_payment,liquidated_with_credit,active,saving_paused
1,3116487373,1084938,negotiation,negotiation,active,saving_paused
2,3117811802,10427,negotiation,negotiation,active,saving_paused
3,3114962398,706395,negotiation,negotiation,active,saving_paused
4,3138820144,860408,client_settled_outside,negotiation,active,program_opening


#Credito

In [15]:
import os, json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =========================
# CARGAR MI_JSON (Colab o GitHub/local)
# =========================
mi_json = None

# 1) Intentar Colab (si está disponible)
try:
    from google.colab import userdata
    mi_json = userdata.get("MI_JSON")
except Exception:
    pass

# 2) Si no es Colab, intentar variable de entorno
if not mi_json:
    mi_json = os.environ.get("MI_JSON")

if not mi_json:
    raise ValueError(
        "No se encontró MI_JSON. "
        "En Colab: configura el secreto MI_JSON. "
        "En GitHub/local: define la variable de entorno MI_JSON."
    )

mi_json_dict = json.loads(mi_json)

# =========================
# AUTENTICACIÓN
# =========================
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]

creds = Credentials.from_service_account_info(mi_json_dict, scopes=SCOPES)
client = gspread.authorize(creds)

# =========================
# ABRIR SHEET Y HOJA
# =========================
spreadsheet_id = "1yAwiQ72vfiSFpoObCQdHWFW3BmARLS-uLl4vc1dN5EY"
sheet_name = "Priorización Batch"

ws = client.open_by_key(spreadsheet_id).worksheet(sheet_name)

# =========================
# LEER COMO DATAFRAME
# =========================
data = ws.get_all_records()
df_batch = pd.DataFrame(data)

df_batch.head()

,Referencia,Origen,Vehículo,Linea de Negocio,Pricing,Tipo de Cl.,AM,AMDUM,Status Reparadora,DR TOTAL,...,DE ($),DE (#),Edad,oct-25,nov-25,dic-25,NO_PERFILA,Criterio,DEMANDAS,Potencial
0,3113739685,Batch 1,POWWI,BRAVO_CO,NORMAL,13,559571,1.45%,Activo,38677800,...,8389053,14,27,NO_OPP,NO_OPP,NO_OPP,FALSE,FALSE,0,14
1,3208532700,Batch 1,POWWI,BRAVO_CO,NORMAL,13,162219,2.35%,Activo,6891150,...,0,0,52,NO_OPP,NO_OPP,NO_OPP,FALSE,FALSE,0,0
2,3114382060,Batch 1,COINK,BRAVO_CO,NORMAL,13,297337,2.10%,Activo,14128800,...,1446068,4,51,NO_OPP,NO_OPP,NO_OPP,FALSE,FALSE,0,4
3,3115731117,Batch 1,COINK,BRAVO_CO,NORMAL,13,326303,2.84%,Baja,11483568,...,0,0,34,NO_OPP,NO_OPP,NO_OPP,FALSE,FALSE,0,0
4,3156021346,Batch 1,COINK,BRAVO_CO,NORMAL,13,232885,2.97%,Baja,7843299,...,0,0,31,NO_OPP,NO_OPP,NO_OPP,FALSE,FALSE,0,0


In [16]:
import os, json
import pandas as pd
import gspread
from google.oauth2.service_account import Credentials

# =========================
# CARGAR MI_JSON (Colab o GitHub)
# =========================
mi_json = None

# 1) Intentar Colab (si está disponible)
try:
    from google.colab import userdata  # solo existe en Colab
    mi_json = userdata.get("MI_JSON")
except Exception:
    pass

# 2) Si no es Colab, intentar variable de entorno (GitHub Secrets / local)
if not mi_json:
    mi_json = os.environ.get("MI_JSON")

if not mi_json:
    raise ValueError(
        "No se encontró MI_JSON. "
        "En Colab: configura el secreto MI_JSON. "
        "En GitHub/local: define la variable de entorno MI_JSON."
    )

mi_json_dict = json.loads(mi_json)

# =========================
# AUTENTICACIÓN (gspread)
# =========================
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets.readonly",
    "https://www.googleapis.com/auth/drive.readonly",
]

creds = Credentials.from_service_account_info(mi_json_dict, scopes=SCOPES)
client = gspread.authorize(creds)

# =========================
# ABRIR SHEET Y HOJA
# =========================
spreadsheet_id = "1yAwiQ72vfiSFpoObCQdHWFW3BmARLS-uLl4vc1dN5EY"
sheet_name = "Priorización Novaciones"

ws = client.open_by_key(spreadsheet_id).worksheet(sheet_name)

# =========================
# LEER COMO DATAFRAME
# =========================
data = ws.get_all_records()
df_novaciones = pd.DataFrame(data)

df_novaciones.head()

,Referencia,Origen,Vehículo,Linea de Negocio,Pricing,Tipo de Cl.,Referencia CR,AM,AMDUM,Status Reparadora,...,Edad,oct-25,nov-25,dic-25,NO_PERFILA,DSLP,Dias de Atraso,Criterio,DEMANDAS,Potencial
0,3212343482,Cr Parcial,POWWI,BRAVO_CO,NORMAL,13,8011583,520280,1.95%,Activo,...,34,NO_OPP,NO_OPP,NO_OPP,FALSE,46064,0,TRUE,0,No viable
1,3144626083,Cr Parcial,COINK,BRAVO_CO,NORMAL,13,8008785R,496058,1.87%,Activo,...,30,NO_OPP,NO_OPP,NO_OPP,FALSE,46064,0,TRUE,0,No viable
2,3204352791,Cr Parcial,DING,BRAVO_CO,PRICING 20%,15,8010163AR,1686926,1.27%,Activo,...,52,NO_OPP,NO_OPP,NO_OPP,FALSE,46064,0,TRUE,0,No viable
3,3164667552,Cr Parcial,COINK,BRAVO_CO,NORMAL,13,8011586,581410,1.57%,Activo,...,57,NO_OPP,NO_OPP,NO_OPP,FALSE,46064,0,TRUE,0,No viable
4,3176416662,Cr Parcial,POWWI,BRAVO_CO,NORMAL,13,8011585,728747,1.60%,Activo,...,29,NO_OPP,NO_OPP,NO_OPP,FALSE,46064,0,TRUE,0,No viable


In [17]:
import pandas as pd

# Copias opcionales
df_batch = df_batch.copy()
df_novaciones = df_novaciones.copy()

# 1) Quedarnos solo con las columnas en común
cols_comunes = ["Referencia", "Origen", "Potencial"]

df_batch_sub = df_batch[cols_comunes]
df_novaciones_sub = df_novaciones[cols_comunes]

# 2) Concatenar
df_credi = pd.concat(
    [df_batch_sub, df_novaciones_sub],
    ignore_index=True
)

df_credi

,Referencia,Origen,Potencial
0,3113739685,Batch 1,14
1,3208532700,Batch 1,0
2,3114382060,Batch 1,4
3,3115731117,Batch 1,0
4,3156021346,Batch 1,0
...,...,...,...
4558,,,
4559,,,
4560,,,
4561,,,


Columnas actualizadas

In [18]:
df

,Referencia,Cedula,Nombre del cliente,Id deuda,correo,Deudas Activas,Banco,Número de Crédito,D_BRAVO,DBT,...,Bucket,Negociador,Tipo Elegible,PotCred_num,PotCred_cat,Días Atraso liquidacion,Estado PL,Ultima Liquidacion,dias_desde_ultima_liq,Priority_level
0,3106653868,3763762,Jorge Luis Camargo Fontalvo,899359,jlc5905@gmail.com,6,CREDIVALORES,401093039,3996300.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,1176.0,Atrasado,2025-01-21,380.0,Prioridad 1
1,3106653868,3763762,Jorge Luis Camargo Fontalvo,899360,jlc5905@gmail.com,6,Serfinanza,543000000,5763450.0,85237500.0,...,1,Niyiret Julio Santos,NaN,NaN,NaN,NaN,Al día,2025-01-21,380.0,Prioridad 2
2,3173740082,1034308242,Nailen Josefina Bahamon Sandoval,1255027,nai.bs@hotmail.com,1,Falabella,300002000000,13506762.0,17310912.0,...,0,Lina Maria Peña Gordillo,NaN,NaN,NaN,0.0,Al día,2024-10-25,468.0,Prioridad 1
3,3224821336,1093755862,Diego Andres Delgado Galvis,1275933,diegodlgd90@gmail.com,1,Davivienda,402516857,10257450.0,10257450.0,...,4,Lina Maria Peña Gordillo,NaN,NaN,NaN,196.0,Atrasado,NaT,NaN,Prioridad 2
4,3167086550,37826977,Luz Mery Quintero Blanco,1297740,luzmb.1225@gmail.com,4,Serfinanza,542060188,8083900.0,91913600.0,...,2,Angie Natalia Borda Soler,NaN,NaN,NaN,483.0,Atrasado,2024-04-30,646.0,Prioridad 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41443,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024433,ale.salo.sastoque@gmail.com,5,Serfinanza,543280552,3756500.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2
41444,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024432,ale.salo.sastoque@gmail.com,5,BBVA,620741,948150.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2
41445,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024430,ale.salo.sastoque@gmail.com,5,Sistecredito,82-000907,199500.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2
41446,3104785092,1013662283,CINDY ALEJANDRA SASTOQUE BELTRAN,2024431,ale.salo.sastoque@gmail.com,5,Sistecredito,35-000261,741300.0,9945200.0,...,2,Gabriela Saavedra Latorre,Back,NaN,NaN,0.0,Al día,NaT,NaN,Prioridad 2


In [19]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 41448 entries, 0 to 41447
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   Referencia                41448 non-null  int64         
 1   Cedula                    41444 non-null  object        
 2   Nombre del cliente        41448 non-null  str           
 3   Id deuda                  41448 non-null  int64         
 4   correo                    41448 non-null  str           
 5   Deudas Activas            41448 non-null  int64         
 6   Banco                     41448 non-null  str           
 7   Número de Crédito         41448 non-null  object        
 8   D_BRAVO                   41448 non-null  float64       
 9   DBT                       41448 non-null  float64       
 10  Meses de atraso           41448 non-null  int64         
 11  Dias de Atraso            41448 non-null  int64         
 12  Apartado Mensual          414

In [20]:
# Normalizar nombres por seguridad (opcional pero recomendado)
df.columns = [c.strip() for c in df.columns]

# Columnas que quieres conservar
cols_keep = ["Referencia", "Id deuda", "ID_reparadora"]

# Verificar que existan
missing = set(cols_keep) - set(df.columns)
if missing:
    raise ValueError(f"Faltan columnas en el DataFrame: {missing}")

# Filtrar
df = df[cols_keep].copy()

df.head()

,Referencia,Id deuda,ID_reparadora
0,3106653868,899359,276651
1,3106653868,899360,276651
2,3173740082,1255027,380859
3,3224821336,1275933,386677
4,3167086550,1297740,392720


In [21]:
df

,Referencia,Id deuda,ID_reparadora
0,3106653868,899359,276651
1,3106653868,899360,276651
2,3173740082,1255027,380859
3,3224821336,1275933,386677
4,3167086550,1297740,392720
...,...,...,...
41443,3104785092,2024433,590622
41444,3104785092,2024432,590622
41445,3104785092,2024430,590622
41446,3104785092,2024431,590622


In [22]:
df_saldos_concat

,REFERENCIA,Ahorro,VEHICULO,Saldo
0,3001033679,214,POWWI,258825.0
1,3001220616,204,POWWI,201716.0
2,3001342492,1595433,POWWI,NaN
3,3001456624,0,POWWI,NaN
4,3001542338,0,POWWI,396063.0
...,...,...,...,...
15028,3052664557,0,COINK,1345126.0
15029,3002481664,0,COINK,NaN
15030,3059347258,0,COINK,NaN
15031,3226855953,0,COINK,NaN


In [23]:
import pandas as pd

# =========================
# 0) Normalización básica
# =========================
df.columns = [c.strip() for c in df.columns]
df_saldos_concat.columns = [c.strip() for c in df_saldos_concat.columns]

df["Referencia"] = df["Referencia"].astype(str)
df_saldos_concat["REFERENCIA"] = df_saldos_concat["REFERENCIA"].astype(str)

n0 = len(df)  # para verificar que no cambie

# =========================
# 1) Colapsar df_saldos_concat a 1 fila por REFERENCIA (evita duplicar filas)
# =========================
# Ahorro = suma de Ahorro por referencia (si tienes varias filas por vehículo)
# Por cobrar = tomar el mismo valor por referencia; si varía, usamos max (o sum si aplica)
aux_saldos = (
    df_saldos_concat
      .groupby("REFERENCIA", as_index=False)
      .agg({
          "Ahorro": "max",   # <- Ahorro total por referencia
          "Saldo": "sum"     # <- Por cobrar (si es único, max=único). Si quieres sum, cámbialo a "sum"
      })
      .rename(columns={
          "REFERENCIA": "Referencia",
          "Ahorro": "Ahorro total",
          "Saldo": "Por cobrar"
      })
)

# =========================
# 2) Merge LEFT (NO agrega filas)
# =========================
df = df.merge(aux_saldos, on="Referencia", how="left", validate="many_to_one")

# =========================
# 3) Ahorro medio = Ahorro total / (# Id deuda únicos por Referencia)
#    (sin merges adicionales -> no cambia filas)
# =========================
deudas_unicas = df.groupby("Referencia")["Id deuda"].transform("nunique")
df["Ahorro medio"] = df["Ahorro total"] / deudas_unicas

# =========================
# 4) Verificación estricta
# =========================
assert len(df) == n0, "❌ Se alteró el número de filas (esto no debería pasar)."

df.head()

,Referencia,Id deuda,ID_reparadora,Ahorro total,Por cobrar,Ahorro medio
0,3106653868,899359,276651,1866636.0,0.0,933318.000000
1,3106653868,899360,276651,1866636.0,0.0,933318.000000
2,3173740082,1255027,380859,NaN,NaN,NaN
3,3224821336,1275933,386677,102285.0,0.0,102285.000000
4,3167086550,1297740,392720,220.0,836230.0,73.333333


In [24]:
df

,Referencia,Id deuda,ID_reparadora,Ahorro total,Por cobrar,Ahorro medio
0,3106653868,899359,276651,1866636.0,0.0,933318.000000
1,3106653868,899360,276651,1866636.0,0.0,933318.000000
2,3173740082,1255027,380859,NaN,NaN,NaN
3,3224821336,1275933,386677,102285.0,0.0,102285.000000
4,3167086550,1297740,392720,220.0,836230.0,73.333333
...,...,...,...,...,...,...
41443,3104785092,2024433,590622,205.0,486410.0,41.000000
41444,3104785092,2024432,590622,205.0,486410.0,41.000000
41445,3104785092,2024430,590622,205.0,486410.0,41.000000
41446,3104785092,2024431,590622,205.0,486410.0,41.000000


In [25]:
import pandas as pd
import numpy as np

# =========================
# 0) Preparación: columnas y tipos
# =========================
# (no cambia filas de df)
df.columns = [c.strip() for c in df.columns]
df_11382.columns = [c.strip() for c in df_11382.columns]

# Claves como string para evitar no-match por tipo
df["Referencia"] = df["Referencia"].astype(str)
df["ID_reparadora"] = df["ID_reparadora"].astype(str)

# En df_11382: bank_reference y message_date_utc (y un id para cruzar por ID_reparadora)
# Normalizamos nombres a minúscula para manejar variantes
df_11382 = df_11382.copy()
df_11382.columns = [c.lower() for c in df_11382.columns]

# Asegurar que existan estas columnas (ajusta si tu df_11382 tiene otro nombre para el id)
required = {"bank_reference", "message_date_utc"}
missing = required - set(df_11382.columns)
if missing:
    raise ValueError(f"En df_11382 faltan columnas requeridas: {missing}")

# Detectar columna id en df_11382 para match con ID_reparadora (ajusta aquí si ya sabes el nombre exacto)
# Preferencias típicas:
id_candidates = ["id_reparadora", "id", "id_deuda", "reparadora_id"]
id_col = next((c for c in id_candidates if c in df_11382.columns), None)
if id_col is None:
    raise ValueError(
        "No encontré en df_11382 una columna de ID para cruzar con ID_reparadora. "
        "Dime el nombre exacto de esa columna en df_11382 (ej: 'id_reparadora')."
    )

# Tipos para llaves
df_11382["bank_reference"] = df_11382["bank_reference"].astype(str)
df_11382[id_col] = df_11382[id_col].astype(str)

# =========================
# 1) Asegurar fecha en hora Colombia y sin tz (datetime normal)
# =========================
df_11382["message_date_utc"] = (
    pd.to_datetime(df_11382["message_date_utc"], utc=True, errors="coerce")
      .dt.tz_convert("America/Bogota")
      .dt.tz_localize(None)
)

# =========================
# 2) Sacar ÚLTIMA fecha por bank_reference y por id (max)
# =========================
last_by_ref = (
    df_11382.groupby("bank_reference", as_index=False)["message_date_utc"]
           .max()
           .rename(columns={"bank_reference": "Referencia", "message_date_utc": "fecha_mensaje_ref"})
)

last_by_id = (
    df_11382.groupby(id_col, as_index=False)["message_date_utc"]
           .max()
           .rename(columns={id_col: "ID_reparadora", "message_date_utc": "fecha_mensaje_id"})
)

# =========================
# 3) Unir a df (SIN agregar filas) y resolver fallback:
#    primero por Referencia, si falta entonces por ID_reparadora
# =========================
n0 = len(df)

df = df.merge(last_by_ref, on="Referencia", how="left", validate="many_to_one")
df = df.merge(last_by_id, on="ID_reparadora", how="left", validate="many_to_one")

# fecha mensaje final (prioriza por referencia)
df["fecha mensaje"] = df["fecha_mensaje_ref"].combine_first(df["fecha_mensaje_id"])

# limpiar auxiliares
df = df.drop(columns=["fecha_mensaje_ref", "fecha_mensaje_id"])

assert len(df) == n0, "❌ Se alteró el número de filas (esto no debería pasar)."

# =========================
# 4) Crear "ultimo contacto" (0 si este mes, 1 si mes pasado, ..., 6 si >6 o NaN)
# =========================
now_co = pd.Timestamp.now(tz="America/Bogota").tz_localize(None)

# diferencia en meses entre (hoy) y (fecha mensaje)
# (year*12 + month) para ambos y restar
msg = df["fecha mensaje"]
months_diff = (now_co.year * 12 + now_co.month) - (msg.dt.year * 12 + msg.dt.month)

# si NaN -> 6; si >6 -> 6; si <0 (fecha futura) -> 0
df["ultimo contacto"] = months_diff.clip(lower=0)
df["ultimo contacto"] = df["ultimo contacto"].where(msg.notna(), 6)
df["ultimo contacto"] = df["ultimo contacto"].clip(upper=6).astype("Int64")

df.head()

,Referencia,Id deuda,ID_reparadora,Ahorro total,Por cobrar,Ahorro medio,fecha mensaje,ultimo contacto
0,3106653868,899359,276651,1866636.0,0.0,933318.000000,2026-01-28 14:46:46,1
1,3106653868,899360,276651,1866636.0,0.0,933318.000000,2026-01-28 14:46:46,1
2,3173740082,1255027,380859,NaN,NaN,NaN,2026-01-23 07:59:29,1
3,3224821336,1275933,386677,102285.0,0.0,102285.000000,2026-01-17 11:13:28,1
4,3167086550,1297740,392720,220.0,836230.0,73.333333,2026-02-08 08:42:30,0


In [26]:
import pandas as pd

# =========================
# 0) Normalización básica
# =========================
df.columns = [c.strip() for c in df.columns]
df_11320.columns = [c.strip() for c in df_11320.columns]

# Clave como string para evitar no-match
df["Id deuda"] = df["Id deuda"].astype(str)              #R
df_11320["id_deuda"] = df_11320["id_deuda"].astype(str) #r

n0 = len(df)  # para verificación final

# =========================
# 1) Colapsar df_11320 a 1 fila por referencia
#    (tomamos el último estado válido; si prefieres otro criterio, dime)
# =========================
aux_estado = (
    df_11320
      .sort_values(by=["id_deuda"])  # orden neutro (no afecta max)
      .groupby("id_deuda", as_index=False)
      .agg({
          "estado_deuda": "last",
          "sub_estado_deuda": "last",
          "estado_reparadora": "last",
          "sub_estado_reparadora": "last"
      })
      .rename(columns={"id_deuda": "Id deuda"})
)

# =========================
# 2) Merge LEFT (NO agrega filas)
# =========================
df = df.merge(
    aux_estado,
    on="Id deuda",
    how="left",
    validate="many_to_one"   # 🔒 evita duplicación silenciosa
)

# =========================
# 3) Verificación estricta
# =========================
assert len(df) == n0, "❌ Se alteró el número de filas (esto no debería pasar)."

df.head()

,Referencia,Id deuda,ID_reparadora,Ahorro total,Por cobrar,Ahorro medio,fecha mensaje,ultimo contacto,estado_deuda,sub_estado_deuda,estado_reparadora,sub_estado_reparadora
0,3106653868,899359,276651,1866636.0,0.0,933318.000000,2026-01-28 14:46:46,1,negotiation,negotiation,active,saving_paused
1,3106653868,899360,276651,1866636.0,0.0,933318.000000,2026-01-28 14:46:46,1,negotiation,negotiation,active,saving_paused
2,3173740082,1255027,380859,NaN,NaN,NaN,2026-01-23 07:59:29,1,negotiation,negotiation,active,saving_paused
3,3224821336,1275933,386677,102285.0,0.0,102285.000000,2026-01-17 11:13:28,1,negotiation,negotiation,active,saving_paused
4,3167086550,1297740,392720,220.0,836230.0,73.333333,2026-02-08 08:42:30,0,negotiation,negotiation,active,saving_paused


In [27]:
import pandas as pd

# =========================
# 0) Normalización básica
# =========================
df.columns = [c.strip() for c in df.columns]
df_credi.columns = [c.strip() for c in df_credi.columns]

df["Referencia"] = df["Referencia"].astype(str)
df_credi["Referencia"] = df_credi["Referencia"].astype(str)

n0 = len(df)  # verificación final

# =========================
# 1) Colapsar df_credi a 1 fila por Referencia
# =========================
aux_credi = (
    df_credi
      .groupby("Referencia", as_index=False)
      .agg({
          "Origen": "last",        # categoría → último valor
          "Potencial": "last"      # valor → último (o max si prefieres)
      })
      .rename(columns={
          "Origen": "tipo de Liquidacion",
          "Potencial": "Potencial Credito"
      })
)

# =========================
# 2) Merge LEFT (NO agrega filas)
# =========================
df = df.merge(
    aux_credi,
    on="Referencia",
    how="left",
    validate="many_to_one"   # 🔒 evita duplicaciones
)

# =========================
# 3) Verificación estricta
# =========================
assert len(df) == n0, "❌ Se alteró el número de filas (esto no debería pasar)."

df.head()

,Referencia,Id deuda,ID_reparadora,Ahorro total,Por cobrar,Ahorro medio,fecha mensaje,ultimo contacto,estado_deuda,sub_estado_deuda,estado_reparadora,sub_estado_reparadora,tipo de Liquidacion,Potencial Credito
0,3106653868,899359,276651,1866636.0,0.0,933318.000000,2026-01-28 14:46:46,1,negotiation,negotiation,active,saving_paused,NaN,NaN
1,3106653868,899360,276651,1866636.0,0.0,933318.000000,2026-01-28 14:46:46,1,negotiation,negotiation,active,saving_paused,NaN,NaN
2,3173740082,1255027,380859,NaN,NaN,NaN,2026-01-23 07:59:29,1,negotiation,negotiation,active,saving_paused,NaN,NaN
3,3224821336,1275933,386677,102285.0,0.0,102285.000000,2026-01-17 11:13:28,1,negotiation,negotiation,active,saving_paused,NaN,NaN
4,3167086550,1297740,392720,220.0,836230.0,73.333333,2026-02-08 08:42:30,0,negotiation,negotiation,active,saving_paused,NaN,NaN


In [28]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 41448 entries, 0 to 41447
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Referencia             41448 non-null  str           
 1   Id deuda               41448 non-null  str           
 2   ID_reparadora          41448 non-null  str           
 3   Ahorro total           31262 non-null  float64       
 4   Por cobrar             31262 non-null  float64       
 5   Ahorro medio           31262 non-null  float64       
 6   fecha mensaje          37455 non-null  datetime64[us]
 7   ultimo contacto        41448 non-null  Int64         
 8   estado_deuda           41226 non-null  str           
 9   sub_estado_deuda       30813 non-null  str           
 10  estado_reparadora      41226 non-null  str           
 11  sub_estado_reparadora  41221 non-null  str           
 12  tipo de Liquidacion    9254 non-null   str           
 13  Potencial Cr

In [29]:
import io
import json
import os
import re
from datetime import datetime
from typing import Dict, List, Optional, Tuple

import pandas as pd
from google.oauth2.service_account import Credentials
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload, MediaIoBaseUpload

import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows


# ======================================================
# 1) Credenciales (Colab / GitHub) — CON ESCRITURA
# ======================================================
def get_credentials() -> Credentials:
    try:
        from google.colab import userdata  # type: ignore
        mi_json = userdata.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado en Colab userdata")
        info = json.loads(mi_json) if isinstance(mi_json, str) else mi_json
        print("Entorno detectado: Google Colab")
    except Exception:
        mi_json = os.environ.get("MI_JSON")
        if not mi_json:
            raise ValueError("MI_JSON no encontrado como variable de entorno (GitHub/local)")
        info = json.loads(mi_json)
        print("Entorno detectado: GitHub / local")

    return Credentials.from_service_account_info(
        info,
        scopes=["https://www.googleapis.com/auth/drive"],
    )

creds = get_credentials()
drive_service = build("drive", "v3", credentials=creds)


# ======================================================
# 2) Helpers: meses / archivos
# ======================================================
MES_MAP = {"ene": 1, "feb": 2, "mar": 3, "abr": 4, "may": 5, "jun": 6, "jul": 7, "ago": 8, "sep": 9, "oct": 10, "nov": 11, "dic": 12}
MES_NOMBRE = {1: "Enero", 2: "Febrero", 3: "Marzo", 4: "Abril", 5: "Mayo", 6: "Junio", 7: "Julio", 8: "Agosto", 9: "Septiembre", 10: "Octubre", 11: "Noviembre", 12: "Diciembre"}

def sheet_name_from_date(dt: datetime) -> str:
    return f"{MES_NOMBRE[dt.month]} {dt.year}"

def parse_range_from_filename(name: str) -> Optional[Tuple[int, int, int, int]]:
    m = re.search(r"([A-Za-z]{3})(\d{2})\s*-\s*([A-Za-z]{3})(\d{2})", name, flags=re.IGNORECASE)
    if not m:
        return None
    m1, y1, m2, y2 = m.group(1).lower(), m.group(2), m.group(3).lower(), m.group(4)
    if m1 not in MES_MAP or m2 not in MES_MAP:
        return None
    return (2000 + int(y1), MES_MAP[m1], 2000 + int(y2), MES_MAP[m2])

def month_index(year: int, month: int) -> int:
    return year * 12 + month

def file_covers_month(file_range: Tuple[int, int, int, int], target_dt: datetime) -> bool:
    sy, sm, ey, em = file_range
    t = month_index(target_dt.year, target_dt.month)
    a = month_index(sy, sm)
    b = month_index(ey, em)
    return a <= t <= b

def is_df_empty_like(df: Optional[pd.DataFrame]) -> bool:
    if df is None or df.shape[0] == 0:
        return True
    return df.dropna(how="all").shape[0] == 0

def shift_month(dt: datetime, n: int) -> datetime:
    y = dt.year + (dt.month - 1 + n) // 12
    m = (dt.month - 1 + n) % 12 + 1
    return datetime(y, m, 1)


# ======================================================
# 3) Buscar el archivo en carpeta, bajar workbook completo
# ======================================================
def list_assignment_files_in_folder(folder_id: str) -> List[Dict]:
    q = f"'{folder_id}' in parents and trashed=false and name contains 'Asignaciones de Cartera'"
    files: List[Dict] = []
    page_token = None
    while True:
        resp = drive_service.files().list(
            q=q, fields="nextPageToken, files(id,name,mimeType,modifiedTime)", pageToken=page_token
        ).execute()
        for f in resp.get("files", []):
            fr = parse_range_from_filename(f.get("name", ""))
            if fr:
                f["parsed_range"] = fr
                files.append(f)
        page_token = resp.get("nextPageToken")
        if not page_token:
            break
    if not files:
        raise ValueError("No encontré archivos 'Asignaciones de Cartera' con rango tipo Ene26-Abr26 en la carpeta.")
    return files

def pick_file_for_month(files_meta: List[Dict], target_dt: datetime) -> Dict:
    candidates = []
    for f in files_meta:
        fr = f["parsed_range"]
        if file_covers_month(fr, target_dt):
            sy, sm, ey, em = fr
            span = month_index(ey, em) - month_index(sy, sm)
            candidates.append((span, f.get("modifiedTime", ""), f))
    if not candidates:
        raise ValueError("No hay archivo que cubra el mes objetivo (revisa nombres/rangos).")
    candidates.sort(key=lambda x: (x[0], x[1]))
    min_span = candidates[0][0]
    same_span = [c for c in candidates if c[0] == min_span]
    same_span.sort(key=lambda x: x[1], reverse=True)
    return same_span[0][2]

def download_file_to_bytes(file_id: str) -> bytes:
    buf = io.BytesIO()
    request = drive_service.files().get_media(fileId=file_id)
    downloader = MediaIoBaseDownload(buf, request)
    done = False
    while not done:
        _, done = downloader.next_chunk()
    return buf.getvalue()

def upload_bytes_overwrite(file_id: str, content: bytes):
    media = MediaIoBaseUpload(
        io.BytesIO(content),
        mimetype="application/vnd.openxmlformats-officedocument.spreadsheetml.sheet",
        resumable=True
    )
    drive_service.files().update(fileId=file_id, media_body=media).execute()


# ======================================================
# 4) PATCH SIN MERGE — con regla especial para Tipo de Liquidacion
# ======================================================
def patch_excel_inplace_by_id_deuda(df_excel: pd.DataFrame, df_actualizado: pd.DataFrame) -> pd.DataFrame:
    df_excel = df_excel.copy()
    df_actualizado = df_actualizado.copy()

    df_excel.columns = [c.strip() for c in df_excel.columns]
    df_actualizado.columns = [c.strip() for c in df_actualizado.columns]

    if "Id deuda" not in df_excel.columns:
        raise ValueError("El Excel no tiene columna 'Id deuda'")
    if "Id deuda" not in df_actualizado.columns:
        raise ValueError("Tu df (actualizado) no tiene columna 'Id deuda'")

    # Crear SOLO las 2 columnas nuevas permitidas si no existen
    for newcol in ["fecha mensaje", "Potencial Credito"]:
        if newcol not in df_excel.columns:
            df_excel[newcol] = pd.NA

    # mapeo df -> excel (NO actualizar ID_reparadora)
    col_map_df = {
        "Ahorro total": "Ahorro total",
        "Por cobrar": "Por cobrar",
        "Ahorro medio": "Ahorro medio",
        "fecha mensaje": "fecha mensaje",
        "ultimo contacto": "ultimo contacto",
        "Estado Deuda": "estado_deuda",
        "sub_estado_deuda": "sub_estado_deuda",
        "estado_reparadora": "estado_reparadora",
        "sub_estado_reparadora": "sub_estado_reparadora",
        "Tipo de Liquidacion": "tipo de Liquidacion",
        "Potencial Credito": "Potencial Credito",
    }

    missing_src = [src for src in col_map_df.values() if src not in df_actualizado.columns]
    if missing_src:
        raise ValueError(
            f"Tu df no tiene estas columnas fuente: {missing_src}\n"
            f"Columnas df: {list(df_actualizado.columns)}"
        )

    # Llaves
    df_excel["Id deuda"] = df_excel["Id deuda"].astype(str).str.strip()
    df_actualizado["Id deuda"] = df_actualizado["Id deuda"].astype(str).str.strip()

    df_excel_idx = df_excel.set_index("Id deuda", drop=False)

    # Fuente (1 fila por Id deuda)
    src_cols = ["Id deuda"] + list(set(col_map_df.values()))
    df_src = (
        df_actualizado[src_cols]
        .drop_duplicates(subset=["Id deuda"], keep="last")
        .set_index("Id deuda")
        .reindex(df_excel_idx.index)  # NO agrega filas
    )

    # 1) Overwrite normal para TODAS excepto Tipo de Liquidacion
    normal_update_cols = [
        "Ahorro total",
        "Por cobrar",
        "Ahorro medio",
        "fecha mensaje",
        "ultimo contacto",
        "Estado Deuda",
        "sub_estado_deuda",
        "estado_reparadora",
        "sub_estado_reparadora",
        "Potencial Credito",
    ]

    for dest in normal_update_cols:
        src = col_map_df[dest]
        df_excel_idx[dest] = df_src[src].values

    # 2) Regla especial para Tipo de Liquidacion:
    #    - Si df trae NaN / vacío => NO actualizar (dejar el valor del archivo)
    dest = "Tipo de Liquidacion"
    src = col_map_df[dest]
    new_vals = df_src[src]

    # vacío si NaN o string vacío/espacios
    mask_keep_old = new_vals.isna() | (new_vals.astype(str).str.strip() == "")

    old_vals = df_excel_idx[dest] if dest in df_excel_idx.columns else pd.Series(index=df_excel_idx.index, dtype="object")
    combined = new_vals.copy()
    combined[mask_keep_old] = old_vals[mask_keep_old]

    df_excel_idx[dest] = combined.values

    out = df_excel_idx.reset_index(drop=True)

    # Reordenar columnas nuevas
    if "ultimo contacto" in out.columns and "fecha mensaje" in out.columns:
        cols = list(out.columns)
        cols.remove("fecha mensaje")
        idx = cols.index("ultimo contacto") + 1
        cols.insert(idx, "fecha mensaje")
        out = out[cols]

    if "MORA_CREDITO" in out.columns and "Potencial Credito" in out.columns:
        cols = list(out.columns)
        cols.remove("Potencial Credito")
        idx = cols.index("MORA_CREDITO") + 1
        cols.insert(idx, "Potencial Credito")
        out = out[cols]

    return out


# ======================================================
# 5) Reemplazar HOJA en el workbook y subir a Drive
# ======================================================
def replace_sheet_with_df(wb: openpyxl.Workbook, sheet_name: str, df_sheet: pd.DataFrame):
    if sheet_name not in wb.sheetnames:
        wb.create_sheet(sheet_name)
    ws = wb[sheet_name]
    ws.delete_rows(1, ws.max_row)

    for r in dataframe_to_rows(df_sheet, index=False, header=True):
        ws.append(r)

def pick_month_sheet_nonempty_from_workbook_bytes(xlsx_bytes: bytes, max_back_months: int = 6) -> Tuple[str, pd.DataFrame, openpyxl.Workbook]:
    wb = openpyxl.load_workbook(io.BytesIO(xlsx_bytes))
    today = datetime.today()

    for back in range(0, max_back_months + 1):
        sheet = sheet_name_from_date(shift_month(today, -back))
        if sheet not in wb.sheetnames:
            continue
        df_try = pd.read_excel(io.BytesIO(xlsx_bytes), sheet_name=sheet, engine="openpyxl")
        if is_df_empty_like(df_try):
            continue
        return sheet, df_try, wb

    raise RuntimeError("No encontré una hoja no vacía en el workbook dentro del rango de meses.")

def update_drive_excel_file(folder_id: str, df_actualizado: pd.DataFrame, max_back_months: int = 6):
    files_meta = list_assignment_files_in_folder(folder_id)
    chosen = pick_file_for_month(files_meta, datetime.today())

    xlsx_bytes = download_file_to_bytes(chosen["id"])
    sheet_used, df_excel_mes, wb = pick_month_sheet_nonempty_from_workbook_bytes(xlsx_bytes, max_back_months=max_back_months)

    print(f"✅ Base encontrada\n   Archivo: {chosen['name']}\n   Hoja:    {sheet_used}")

    df_excel_actualizado = patch_excel_inplace_by_id_deuda(df_excel_mes, df_actualizado)
    replace_sheet_with_df(wb, sheet_used, df_excel_actualizado)

    out_buf = io.BytesIO()
    wb.save(out_buf)
    upload_bytes_overwrite(chosen["id"], out_buf.getvalue())

    print("✅ Archivo actualizado en Drive (sobrescrito).")


# ======================================================
# 6) EJECUCIÓN FINAL
# ======================================================
FOLDER_ID = "1cf2p3R7iM0xowAt4muEruDwxZoZqD_jB"
update_drive_excel_file(folder_id=FOLDER_ID, df_actualizado=df, max_back_months=6)

update_drive_excel_file(folder_id=FOLDER_ID, df_actualizado=df, max_back_months=6)

Entorno detectado: GitHub / local


✅ Base encontrada
   Archivo: Asignaciones de Cartera Ene26-Abr26.xlsx
   Hoja:    Febrero 2026


✅ Archivo actualizado en Drive (sobrescrito).


✅ Base encontrada
   Archivo: Asignaciones de Cartera Ene26-Abr26.xlsx
   Hoja:    Febrero 2026


✅ Archivo actualizado en Drive (sobrescrito).
